# Hakai Nutrient QA-QC 
---
# Start Notebook
## Install packages 
Google colab servers has already a few commonly used packages installed. To install those missing, which are specific toh Hakai let's run the following commands:

In [ ]:
!pip install hakai-api
!pip install git+git://github.com/HakaiInstitute/ioos_qc.git@development
!pip install git+git://github.com/HakaiInstitute/process_ocean_timeseries

  Cloning git://github.com/HakaiInstitute/ioos_qc.git (to revision development) to /tmp/pip-req-build-fe4277jw
  Running command git clone -q git://github.com/HakaiInstitute/ioos_qc.git /tmp/pip-req-build-fe4277jw
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
  Created wheel for ioos-qc: filename=ioos_qc-1.0.0-cp37-none-any.whl size=44537 sha256=f28c25bb1443908da30507e33a932dcbc9a24e55241d91dd1102e7b581a1fb5e
  Stored in directory: /tmp/pip-ephem-wheel-cache-f4v3h38o/wheels/2f/bb/7b/19f2997af03515525ae518c26716a8cc13c33e4be7bbd0b344
Successfully built ioos-qc
  Cloning git://github.com/HakaiInstitute/process_ocean_timeseries to /tmp/pip-req-build-8dpvrkpo
  Running command git clone -q git://github.com/HakaiInstitute/process_ocean_timeseries /tmp/pip-req-build-8dpvrkpo
  Created wheel for process-ocean-data: filename=process_ocean_data-0.

## Import the python packages needed

In [ ]:
# Let's load pandas for working with the data in table
import pandas as pd 
import numpy as np

# Let's load seaborn to plot the data
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# This to install the hakai api tool and be able to download some data from hakai's database
from hakai_api import Client

# Install ioos_qc which is used to qc data
from ioos_qc.config import QcConfig
from ioos_qc import qartod

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets, HBox, VBox
import time

from process_ocean_data.tools import qc as review

# Define some project standards

In [ ]:
# Flag convention
flag_convention = 'HAKAI'
flag_color_map = {key:value['Color'] for key,value in review.flag_conventions[flag_convention].items()}

# And the interesting variables to use from the Hakai database
nutrient_variables = ['no2_no3_um','sio2','po4']

pd.options.display.max_columns = 100

# Download data from the Hakai database
For more information regarding the Hakai API, go [here](https://github.com/HakaiInstitute/hakai-api).

You can find a list of all the data type endpoints [here](http://hakaiinstitute.github.io/hakai-api/#endpoints).

In [ ]:
# Let's retrieve the endpoint to retrieve nutrients data:
endpointUrl = '/eims/views/output/nutrients'
site_id = 'QU39'
start_time = '2015-01-01'
end_time = '2022-06-11'
# We'll retrieve data only associated with QU39 between January 1st 2019 to January 1st 2020
filterUrl = 'site_id={0}&collected>{1}&collected<{2}&limit=-1'.format(
    site_id, start_time, end_time
)

# Get Hakai Data    
#Get Data from Hakai API
client = Client() # Follow stdout prompts to get an API token

# Make a data request for sampling stations
url = '%s/%s?%s' % (client.api_root,endpointUrl,filterUrl)
response = client.get(url)
df = pd.DataFrame(response.json())
original_columns = df.columns
print(str(len(df))+' records downloaded')
df.head()

3269 records downloaded


,action,event_pk,rn,is_replicate,date,work_area,organization,survey,sampling_bout,site_id,project_specific_id,hakai_id,source,lat,long,gather_lat,gather_long,collection_method,line_out_depth,pressure_transducer_depth,filtered,filter_type,volume,installed,collected,preserved,analyzed,lab_technician,nh4_,no2_no3_um,no2_no3_ugl,no2_no3_units,tp,tdp,tn,tdn,srp,po4,sio2,po4pfilt,no3nfilt,po4punfl,no3nunfl,nh4nunfl,nh4__flag,no2_no3_flag,tp_flag,tdp_flag,tn_flag,tdn_flag,srp_flag,po4_flag,sio2_flag,po4pfilt_flag,no3nfilt_flag,po4punfl_flag,no3nunfl_flag,nh4nunfl_flag,analyzing_lab,row_flag,metadata_qc_flag,quality_level,comments,quality_log
0,,416,1,None,2015-03-18,QUADRA,HAKAI,"QOMA,QOMA1,QOMA2",1,QU39,QNUT382,QNUT382,M,50.0307,-125.0992,NaN,NaN,None,0,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,5.918382,None,uM,None,None,None,None,None,0.637,12.324552,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Checked by KP\r2: Every sample taken today ...
1,,416,1,None,2015-03-18,QUADRA,HAKAI,"QOMA,QOMA1,QOMA2",1,QU39,QNUT383,QNUT383,M,50.0307,-125.0992,NaN,NaN,None,5,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,6.011794,None,uM,None,None,None,None,None,0.638,12.396646,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Checked by KP\r2: Every sample taken today ...
2,,416,1,None,2015-03-18,QUADRA,HAKAI,"QOMA,QOMA1,QOMA2",1,QU39,QNUT384,QNUT384,M,50.0307,-125.0992,NaN,NaN,None,10,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,6.634283,None,uM,None,None,None,None,None,0.700,12.694025,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Checked by KP\r2: Every sample taken today ...
3,,416,1,None,2015-03-18,QUADRA,HAKAI,"QOMA,QOMA1,QOMA2",1,QU39,QNUT385,QNUT385,M,50.0307,-125.0992,NaN,NaN,None,30,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,24.151575,None,uM,None,None,None,None,None,2.085,42.222115,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Checked by KP\r2: Every sample taken today ...
4,,416,1,None,2015-03-18,QUADRA,HAKAI,"QOMA,QOMA1,QOMA2",1,QU39,QNUT386,QNUT386,M,50.0307,-125.0992,NaN,NaN,None,50,NaN,None,0.45nm,13,None,2015-03-18T16:02:55.000Z,2015-03-18T16:02:14.000Z,2015-05-21T07:00:00.000Z,"Kate,Katie,Rebecca",None,26.960609,None,uM,None,None,None,None,None,2.251,49.347595,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,UBC,Results,None,Technicianm,,1: Checked by KP\r2: Every sample taken today ...


In [ ]:
# Let's convert the collected time to a datetime object variable called  time 
#  and extract the from those datetime objects the year and month 
df['time'] = pd.to_datetime(df['collected'])
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['dayoftheyear'] = pd.to_timedelta(df['time'].dt.dayofyear, unit='d')

# Define a depth variable which is: 
#   - pressure_transducer_depth (if available)
#   - OR line_out_depth
df['depth'] = df['pressure_transducer_depth'].fillna(df['line_out_depth'])

# Review Replicates
## Pool Standard Deviation


In [ ]:
 # Let's create a pooled standard deviation function
def pooled_standard_deviation(df_to_review,count_col='count',std_col='std'):
    # Keep only records that have replicates
    df_replicates = df_to_review[df_to_review[count_col]>1]
    upper =  df_replicates[count_col].sub(-1).mul(df_replicates[std_col].pow(2)).sum()
    lower = df_replicates[count_col].sub(-1).sum()
    pooled_std = np.sqrt(upper/lower)
    return pooled_std

In [ ]:
df_grouped = df.groupby(['site_id','line_out_depth','collected']).agg(['mean','std','count'])

for var in nutrient_variables:
    pool_std = pooled_standard_deviation(df_grouped[var])
    print('{0} pool.std.: {1}'.format(var,pool_std))

no2_no3_um pool.std.: 0.12386840168403754
sio2 pool.std.: 0.5301664184882673
po4 pool.std.: 0.03475953788519301


## Replicates Standard Deviation Distribution
Present the distribution of the standard deviations the replicate samples for each line_out_depth.

In [ ]:
px.histogram(df_grouped['po4'].reset_index().dropna(subset=['std']).sort_values('line_out_depth'),
             x='std',color='line_out_depth', hover_name='collected',
             marginal='box')

# Apply detection limit flag
Samples with values lower than the dectection limit will be flag as BDL.



In [ ]:
# If lower than detection limit, flag as BDL
df.loc[df['no2_no3_um']<0.036 ,'no2_no3_um_bdl_flag']='BDL'
df.loc[df['po4']<0.032 ,'po4_bdl_flag']='BDL'
df.loc[df['sio2']<0.1 ,'sio2_bdl_flag']='BDL'

# Time series QARTOD tests
Review each depth time series for a station and run timeseries specific test on them. The configuarion dictionary below, list of each variables and depth ranges the tests that will be applied.


## Set QARTOD Tests Configuration

In [ ]:
qc_config = [{
    "depth_range":{
        "minimum":-5,
        "maximum":55
        },
    "streams":{
        "no2_no3_um":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 36],
                    "fail_span": [0, 40]
                    },
                "aggregate": {}
            }
        },
        "po4":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 3],
                    "fail_span": [0, 4]
                    },
                "aggregate": {}
            }
                    
        },
        "sio2":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 80],
                    "fail_span": [0, 100]
                    },
                "aggregate": {}
            }
                    
        }
    }
},{
    "depth_range":{
        "minimum":55,
        "maximum":300
        },
    "streams":{
        "no2_no3_um":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 36],
                    "fail_span": [0, 40]
                    },
                "spike_test": {
                    "suspect_threshold": 2,
                    "fail_threshold": 3,
                    "method": "differential",
                    "n_dev_suspect":1,
                    "n_dev_fail":2,
                    "n_records":40,
                    "min_records":20
                    },
                "aggregate": {}
            }
        },
        "po4":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 3],
                    "fail_span": [0, 4]
                    },
                "spike_test": {
                    "suspect_threshold": 0.5,
                    "fail_threshold": 1,
                    "method": "differential",
                    "n_dev_suspect":2,
                    "n_dev_fail":3,
                    "n_records":50,
                    "min_records":40
                    },
                "aggregate": {}
            }
                    
        },
        "sio2":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 80],
                    "fail_span": [0, 100]
                    },
                "spike_test": {
                    "suspect_threshold":8,
                    "fail_threshold": 12,
                    "method": "differential",
                    "n_dev_suspect":2,
                    "n_dev_fail":3,
                    "n_records":30,
                    "min_records":20
                    },
                "aggregate": {}
            }
                    
        }
    }
}]


## Run QARTOD Tests

In [ ]:
# Run QARTOD tests
# We are using the deprecated QcConfig method and hopefully will move 
#  to a new stream method soon.
time = 'time'
depth = 'depth'
group_timeseries = ['line_out_depth']
for item in qc_config:
    df_depth_range = df[(df['depth']>item['depth_range']["minimum"]) & \
                        (df['depth']<item['depth_range']["maximum"])]
    for line_out_depth, timeserie in df_depth_range.groupby(['site_id','line_out_depth']):
        timeserie = timeserie.sort_values(time)
        for var in item['streams'].keys():
            qc = QcConfig(item['streams'][var])
            qc_result = qc.run(
                inp=timeserie[var],
                tinp=timeserie[time],
                zinp=timeserie[depth],
            )
            for module,tests in qc_result.items():
                for test, flag in tests.items():
                    df.loc[timeserie.index, var+'_'+module+"_"+test] = flag

# Map QARTOD Flags to Hakai Convention
for var in df.filter(like='qartod').columns:
     df[var] = df[var].replace({pd.NA:9}).astype(int).replace(review.flag_conventions['mapping']['QARTOD-HAKAI'])

In [ ]:
df.filter(like='qartod').replace({pd.NA:9})

,no2_no3_um_qartod_gross_range_test,no2_no3_um_qartod_aggregate,po4_qartod_gross_range_test,po4_qartod_aggregate,sio2_qartod_gross_range_test,sio2_qartod_aggregate,no2_no3_um_qartod_spike_test,po4_qartod_spike_test,sio2_qartod_spike_test
0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,9.0,9.0
1,1.0,1.0,1.0,1.0,1.0,1.0,9.0,9.0,9.0
2,1.0,1.0,1.0,1.0,1.0,1.0,9.0,9.0,9.0
3,1.0,1.0,1.0,1.0,1.0,1.0,9.0,9.0,9.0
4,1.0,1.0,1.0,1.0,1.0,1.0,9.0,9.0,9.0
...,...,...,...,...,...,...,...,...,...
3264,9.0,2.0,9.0,2.0,9.0,2.0,2.0,2.0,2.0
3265,9.0,2.0,9.0,2.0,9.0,2.0,2.0,2.0,2.0
3266,9.0,2.0,9.0,2.0,9.0,2.0,2.0,2.0,2.0
3267,9.0,2.0,9.0,2.0,9.0,2.0,2.0,2.0,2.0


## Review QARTOD Results

In [ ]:
flag_color_map = {key:value['Color'] for key,value in review.flag_conventions['HAKAI'].items()}

var = 'sio2'
line_out_depth= [100]
flag_var = df.filter(regex='flag|qartod').columns.tolist()
px.scatter(df[df['line_out_depth'].isin(line_out_depth)].sort_values(['line_out_depth',var+'_qartod_aggregate']),
             x='time',y=var,
             color=var+'_qartod_aggregate', 
             hover_data=['hakai_id']+flag_var,
             color_discrete_map=flag_color_map)

# Red field ratio

In [ ]:
max_depth = 50
figs=px.scatter(df[df['line_out_depth']<max_depth],
               x='po4',
               y='no2_no3_um',
               color='line_out_depth', 
               hover_data=['hakai_id','date'], 
               template='simple_white',
               title='PO4',
               labels={'po4':'PO4 (uM)', 'line_out_depth':'Bottle Target Depth (m)'},facet_col='year')
kk=1
line_out_depths = df[df['line_out_depth']<max_depth]['line_out_depth'].unique()
while kk<=len(line_out_depths)+1:
  figs.add_trace(go.Scatter(x = [0, 3],
                            y =  [0, 48],
                            mode='lines',
                            line_color='red',
                            showlegend=False
                            ),
                row=1,
                col=kk)
  kk+=1
figs.show()

# Review Interannual Variability
Let's compute the average value measured for each depth and the associated standard deviation.

## Compute the seasonal variability
### Monthly Values


In [ ]:
# Compute the monthly value recorded for each station and line_out_depth 
df_seasonal = df.groupby(['site_id','line_out_depth','month']) \
  .agg(['mean','std'])[nutrient_variables] \
  .reset_index()
df_seasonal.columns = ['_'.join(filter(None,col)).strip() for col in df_seasonal.columns.values]

# Add a seasonal value to each data
df_with_seasons = pd.merge(df,df_seasonal,on=['site_id','line_out_depth','month'], suffixes=('','_seasonal'))


### Over 60 days window

In [ ]:
# TODO not that pandas.rolling isn't compatible with center window yet.
# Or get the season with a 30 days running window instead
window = '60d'
df_running_seasonal = df.groupby(['site_id','line_out_depth']).apply(
    lambda x: x.sort_values('dayoftheyear').set_index('dayoftheyear')[nutrient_variables]\
       .rolling(window).agg(['mean','std']).reset_index()
)
df_running_seasonal.columns = ['_'.join(filter(None,col)).strip() for col in df_running_seasonal.columns.values]

# Since rolling is looking the days prior, we'll center the value to the middle of the window.
#   rolling(center) isn't yet available for datetime index
df_running_seasonal['dayoftheyear'] = df_running_seasonal['dayoftheyear']-pd.to_timedelta(window)/2
df_running_seasonal.loc[df_running_seasonal['dayoftheyear']<pd.to_timedelta('0d'),'dayoftheyear'] += pd.to_timedelta('365d')

# Clean the matrix a bit
df_running_seasonal = df_running_seasonal.reset_index()\
    .drop('level_2',axis='columns')\
    .set_index(['site_id','line_out_depth','dayoftheyear'])\
    .add_suffix('_seasonal')

# Add a seasonal value to each data
df = pd.merge(df,df_running_seasonal,
                           on=['site_id','line_out_depth','dayoftheyear'])


## Make Monthly Box Plots

In [ ]:
review_depth = 20
var = 'sio2'

# Show the seasonality of the data
px.box(df.sort_values('line_out_depth'),x='month',y='no2_no3_um',
        color='line_out_depth', animation_frame='line_out_depth',
       hover_name='hakai_id')

In [ ]:
# Review each depth timeseries and
review_depth = 10
alpha = 2
var = 'no2_no3_um'

#  compare seasonal average to value recorded
for var_name in ['no2_no3_um','po4','sio2']:
    df[var_name+'_seasonal_flag'] = 'AV'

    residual = (df[var_name]-df[var_name+'_mean_seasonal']).abs()
    df.loc[residual> alpha*df[var_name+'_std_seasonal'], var_name+'_seasonal_flag'] = 'SVC'

# Isolate the review depth
df_temp = df[df['line_out_depth']==review_depth].sort_values('time')

# Plot data
fig = px.scatter(df_temp,x='time',y=var,color=var+'_seasonal_flag', color_discrete_map=flag_color_map,
                 hover_name='hakai_id')

fig.add_trace(go.Scatter(x=df_temp['time'],
                         y=df_temp[var+"_mean_seasonal"],name='running mean',
              line = dict(color='red', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=df_temp['time'],
                         y=df_temp[var+"_mean_seasonal"]+alpha*df_temp[var+"_std_seasonal"],
                         name='running mean+{0}*std'.format(alpha),
              line = dict(color='black', width=2, dash='dot')))
fig.add_trace(go.Scatter(x=df_temp['time'],
                         y=df_temp[var+"_mean_seasonal"]-alpha*df_temp[var+"_std_seasonal"],
                         name='running mean-{0}*std'.format(alpha),
                         line = dict(color='black', width=2, dash='dot')))

fig.update_layout(coloraxis_showscale=False)
fig.show()

# Create Aggregated Suggested Flag


In [ ]:
flags_considered = ['_qartod_aggregate','_bdl_flag']
for var in nutrient_variables:
    flag_vars = [var+item for item in flags_considered]
    df[var+'_review_flag'] = \
       df[flag_vars].apply(lambda x: review.compare_flags(list(x),convention='HAKAI'),axis='columns')

# Update Original Flags
df['po4_flag'] = df['po4_flag'].fillna(df['po4_review_flag'])
df['sio2_flag'] = df['sio2_flag'].fillna(df['sio2_review_flag'])
df['no2_no3_flag'] = df['no2_no3_flag'].fillna(df['no2_no3_um_review_flag'])

# Review Results Manually

In [ ]:
# Plot profile
group_profile_by = 'date'
profile_list = widgets.SelectMultiple(
    options=df[group_profile_by].sort_values().drop_duplicates(),
    value=(df[group_profile_by].iloc[0],),
    description="Select Profiles",
    disabled=False,
)
def make_profile_plot(profiles):
    # Filter only selected profiles
    df_selected = df[df['date'].isin(profiles)]
    df_temp = df[df['date'].isin(profile_list.value)].sort_values(['date','depth'])
    fig = []
    for ii in [0,1,2]:
        fig.append(px.line(df_temp,x=nutrient_variables[ii],y='depth',color='date',hover_data=['hakai_id']))

    subfig = make_subplots(rows=1, cols=3)
    kk=1
    for item in fig:
        for subitem in item.data:
            if kk>1:
                subitem.showlegend=False
            subfig.add_trace(subitem,row=1,col=kk)
        subfig.update_xaxes(title_text=nutrient_variables[kk-1], row=1, col=kk)
        kk += 1
    subfig.update_yaxes(title_text='Depth (m)', row=1, col=1)
    subfig.update_yaxes(autorange="reversed")
    subfig.update_xaxes(matches=None)
    subfig.update_traces(mode='markers+lines')
    return subfig

interact(make_profile_plot, profiles=profile_list)

interactive(children=(SelectMultiple(description='Select Profiles', index=(0,), options=('2015-03-18', '2015-0…

<function __main__.make_profile_plot>

In [ ]:
group_timeseries_by = 'line_out_depth'
# Plot TimeSeries
timeseries_id = widgets.SelectMultiple(
    options=df[group_timeseries_by].sort_values().drop_duplicates().values,
    value=(df[group_timeseries_by].iloc[0],),
    description="Select "+group_timeseries_by,
    disabled=False,
)

nutrient_id = widgets.ToggleButtons(
    options=nutrient_variables,
    value=nutrient_variables[0],
    description="Select "+group_timeseries_by,
    disabled=False,
)
timeseries_id
def plot_timeseries(nutrient, profile):
    flag_columns = df.filter(regex= nutrient+'.*(flag|qartod)').columns
    df_temp = df[df['line_out_depth'].isin(timeseries_id.value)].sort_values('time').copy()
    df_temp[df_temp.filter(like='flag').columns] = df_temp.filter(like='flag').astype(str)
    df_good = df_temp.loc[df_temp[nutrient.replace('_um','')+'_flag']=='AV']
    fig = px.line(df_good,
            x='time',
            y=nutrient,
            line_dash='line_out_depth',
            color=nutrient+'_review_flag',
            color_discrete_map={value:item['Color'] for value, item in review.flag_conventions['HAKAI'].items()}
           )
    fig2 = px.scatter(df_temp,
            x='time',
            y=nutrient,
            color=nutrient+'_review_flag',
            hover_data=['date','hakai_id','depth']+list(flag_columns),
            color_discrete_map={value:item['Color'] for value, item in review.flag_conventions['HAKAI'].items()}
           )
    for item in fig2.data:
        fig.add_trace(item)
    return fig
    
interact(plot_timeseries,nutrient=nutrient_id, profile=timeseries_id)

In [ ]:
df.loc[df[['no2_no3_flag','sio2_flag','po4_flag']].isin(['SVC','SVD','BDL']).any(axis='columns')][['date','hakai_id','no2_no3_flag','sio2_flag','po4_flag']]

In [ ]:
df.filter(regex='_flag$')

In [ ]:
# Review QC Manually
review.manual_qc_interface(df, ['no2_no3','sio2','po4'], 'HAKAI' ,review_flag="_flag")


# Save Result to Hakai Portal Compatible Excel Format

In [ ]:
df_output = df[original_columns]

# Rename Columns to Match Portal Output
# Drop unchanged rows

###ATTENTION### The output has not yet been tested with the Hakai API.
df_output.to_excel('Hakai_Nutrient_Revision_{0}.xlsx'.format(pd.Timestamp.now().isoformat().replace(':','')))


# Report Figures

## Contour plot

In [ ]:

# Plot contourf interpolate linearly over the x axis and maximum over two NaN values
var = 'no2_no3_um'
df_pivot = pd.pivot_table(df,values=var,index='line_out_depth',columns='date',aggfunc='mean').interpolate(axis='index',limit=4).sort_index(axis=0).sort_index(axis=1).interpolate(axis='columns',limit=3)
fig = go.Figure(data =
    go.Contour(z=df_pivot.values,x=df_pivot.columns,y=df_pivot.index.values,
               colorbar=dict(title=var, titleside='right'),
               colorscale='RdYlGn',
               ncontours=10,
               contours_coloring='heatmap'
               #,connectgaps=True
              ))
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside',showline=True)
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()

In [ ]:
df_pivot

date,2015-03-18,2015-03-24,2015-03-31,2015-04-06,2015-04-14,2015-04-22,2015-05-01,2015-05-05,2015-05-13,2015-05-21,2015-05-26,2015-06-03,2015-06-15,2015-06-24,2015-06-30,2015-07-07,2015-07-22,2015-07-28,2015-08-05,2015-08-12,2015-08-20,2015-08-25,2015-09-02,2015-09-09,2015-09-15,2015-09-22,2015-09-28,2015-10-06,2015-10-14,2015-10-20,2015-10-27,2015-11-03,2015-11-10,2015-11-18,2015-11-25,2015-12-11,2015-12-16,2016-01-05,2016-01-13,2016-01-19,2016-01-28,2016-02-02,2016-02-09,2016-02-16,2016-02-23,2016-03-15,2016-03-22,2016-03-29,2016-04-01,2016-04-04,...,2019-03-05,2019-03-12,2019-03-20,2019-03-27,2019-04-02,2019-04-10,2019-05-02,2019-05-09,2019-05-22,2019-05-29,2019-06-04,2019-06-20,2019-06-25,2019-07-03,2019-07-09,2019-07-13,2019-07-18,2019-07-24,2019-07-30,2019-08-07,2019-08-15,2019-08-20,2019-08-29,2019-09-04,2019-09-19,2019-09-24,2019-10-09,2019-10-18,2019-10-30,2019-11-05,2019-11-13,2019-11-20,2019-11-26,2019-12-03,2019-12-13,2020-01-08,2020-01-28,2020-02-05,2020-02-18,2020-02-26,2020-03-06,2020-03-12,2020-03-18,2020-04-17,2020-04-29,2020-05-14,2020-05-26,2020-06-04,2020-06-11,2020-06-17
line_out_depth,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5.918382,12.621053,15.233539,13.544474,1.051300,1.306241,0.174630,0.259875,0.0370,0.5230,0.1080,1.0910,15.048047,0.397398,0.435336,13.255281,11.746085,0.425398,14.811890,0.164206,12.039889,0.834763,5.471024,0.235572,20.734953,12.739000,13.931521,3.6250,13.688164,17.372081,22.2290,18.401000,19.067727,24.735635,26.710776,27.617523,27.537574,29.131010,28.285861,28.493337,28.982707,28.327118,27.624909,27.518331,26.443037,25.890247,23.583460,23.555910,11.919044,0.282179,...,25.863953,13.464348,12.219116,1.263961,0.470154,9.406712,12.645969,19.129772,0.318619,0.067467,0.829620,16.439285,0.038544,0.070001,0.129765,0.069190,1.428770,0.016009,10.809549,0.360385,0.288813,0.201999,11.082626,0.031442,4.735217,3.189728,14.587262,16.742734,21.264775,18.146666,18.039302,20.785129,22.113778,24.044645,23.970383,27.665334,27.526906,27.331394,26.110316,25.920811,25.391200,25.938524,10.948603,14.733008,1.516176,0.074270,3.536981,0.173488,3.381952,0.147706
5,6.011794,12.610005,14.797996,13.311217,1.008608,1.595480,0.781370,0.947949,0.0000,0.0000,0.1520,0.0000,15.843053,0.016930,0.422605,16.388544,11.113477,0.354456,14.875615,0.184426,14.040211,5.162968,5.625918,0.149520,21.691290,14.089739,14.796839,7.1900,13.803899,17.487816,23.6170,18.467644,19.386764,24.422603,26.965340,27.741198,27.139538,29.295941,28.329009,28.463998,28.862724,27.801570,28.080714,27.344202,26.949985,26.035659,24.447000,23.733436,0.696283,0.235350,...,25.750311,13.542952,13.816564,1.618667,0.008967,9.669514,13.242750,21.347497,0.331560,2.128526,3.006384,19.358797,1.110772,0.078848,0.006988,6.427958,1.633636,0.068050,16.682271,1.053226,0.556628,3.208047,17.926657,1.993983,5.594196,6.021573,14.567650,18.491063,21.197503,18.321426,17.654525,20.989983,22.412622,25.016934,24.385708,27.755589,27.494207,27.584451,26.210822,25.819664,25.785579,25.938867,12.712733,15.538813,1.311505,-0.000159,3.424592,0.281775,4.311314,0.093657
10,6.634283,13.773078,15.263013,15.175830,3.358950,16.688783,3.151531,1.277547,2.8790,0.3900,0.3980,0.0520,19.656691,0.358470,17.036152,22.415536,7.393897,1.487425,22.529027,2.873653,20.279494,21.330156,7.027493,2.364463,24.721677,14.413698,18.474739,20.7670,13.486117,17.575108,26.1550,17.556473,20.319711,24.821587,27.066990,27.757324,28.804960,29.258738,28.515290,28.794089,28.867305,28.485010,28.549110,27.579112,27.025928,25.987197,24.947000,25.321216,0.696283,0.983883,...,25.762874,16.482576,18.713931,2.956457,13.601316,11.034110,14.239796,23.901421,0.383331,9.927050,4.993321,22.160544,10.742970,0.380162,0.019963,12.786725,12.289484,13.626225,25.395638,12.569557,13.596884,11.088957,26.673870,15.946035,6.841393,6.938866,14.754410,18.809552,21.862236,18.636208,19.410116,22.049307,22.237451,25.646093,25.058477,27.698091,27.743703,27.622945,27.038911,26.380094,

## Scatter with colorbar

In [ ]:
## Scatter with colorbar
fig = px.scatter(df.dropna(subset=['no2_no3_um'],axis=0),x='time',y='depth',
                 color='no2_no3_um',color_continuous_scale='RdYlGn',
                 hover_name='hakai_id')
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside')
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()